In [1]:
""" MODELO PRE ENTRENADO SwinViT Base
Dataset: Patch Camelyon, de tensorflow preparado para HuggingFace completo
Fuente del modelo: HuggingFace
Modelo: microsoft/swin-sbase-patch4-window7-224
Entrenamiento 3 epocas
"""

' MODELO PRE ENTRENADO SwinViT Base - EXPERIMENTO 2\nIdentificar variación de exactitud respecto a la cantidad de epocas\nDataset: Patch Camelyon, de tensorflow preparado para HuggingFace completo\nFuente del modelo: HuggingFace\nModelo: microsoft/swin-sbase-patch4-window7-224\ne epocas\n'

In [2]:
# Conectar con Google Drivee()
from google.colab import drive
drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"


Mounted at /content/drive


In [3]:
# librerías
!pip install -qqq datasets evaluate keras_cv
!pip install --upgrade transformers -qqq
!pip install -qqq tensorflow
!pip install -qqq tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 61.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import transformers
import evaluate
from transformers import AutoModelForImageClassification, AutoImageProcessor,TrainingArguments, Trainer
from datasets import Dataset as HuggingFaceDataset, Features
from PIL import Image as PILImage
import torch

In [5]:
# importar credenciales de Hugging face
from google.colab import userdata
import os
from huggingface_hub import login

# Accede al token almacenado en los secretos de Colab
hf_token = userdata.get('HF_TOKEN')

## Cargar el dataset patch camelyon de hugging face

In [6]:
train_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/train")

val_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/val")

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
model_name = "microsoft/swin-base-patch4-window7-224"
batch_size = 32

In [ ]:
print(len(train_hf_ds))


262144


### Preprocesar los datos

In [ ]:
num_labels = 2
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes = True
)

# instanciar el image processing
image_processor = AutoImageProcessor.from_pretrained(model_name,do_rescale=True,use_fast=True)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

In [12]:
model_path="microsoft/swin-base-patch4-window7-224"
model = AutoModelForImageClassification.from_pretrained(model_path)
image_processor = AutoImageProcessor.from_pretrained(model_path,do_rescale=True,use_fast=True)
batch_size = 32
num_labels = 2

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

### Métricas

In [9]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    references = p.label_ids
    metrics_result = metric.compute(predictions=predictions, references=references)
    print(f"Metric computation result: {metrics_result}")
    return {"eval_accuracy": metrics_result["accuracy"]}

### Collator

In [10]:
def custom_image_collator(batch):
    # Extract raw images and labels from the batch
    # Assuming example["image"] is already a PIL Image object
    images = [example["image"].convert("RGB") for example in batch]
    labels = [example["label"] for example in batch] # Assuming 'label' is the original label key

    # Process images using the image_processor
    processed_inputs = image_processor(images=images, return_tensors="pt")

    # Stack labels
    stacked_labels = torch.tensor(labels, dtype=torch.long)

    # Return the processed batch
    return {"pixel_values": processed_inputs["pixel_values"], "labels": stacked_labels}

### Parámetros de entrenamiento

In [16]:
# establecer los parámetros del entrenamiento
args = TrainingArguments(
    remove_unused_columns=False,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="none",
    output_dir=base_folder+"/Modelos entrenados/Swin v1 Base 3epocas"
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_hf_ds,
    eval_dataset=val_hf_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_image_collator,
    tokenizer=image_processor
)

/tmp/ipython-input-17-546105683.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
%%time
print("\n--- Iniciando entrenamiento ---")
train_results = trainer.train()


--- Iniciando entrenamiento ---


Epoch,Training Loss,Validation Loss,Accuracy
1,0.087600,0.486094,0.889679
2,0.046200,0.363471,0.905548


Metric computation result: {'accuracy': 0.889678955078125}
Metric computation result: {'accuracy': 0.905548095703125}


Epoch,Training Loss,Validation Loss,Accuracy
1,0.087600,0.486094,0.889679
2,0.046200,0.363471,0.905548
3,0.018500,0.544043,0.903198


Metric computation result: {'accuracy': 0.9031982421875}
CPU times: user 3h 49min 23s, sys: 1h 1min 5s, total: 4h 50min 28s
Wall time: 4h 24min 13s


In [19]:
trainer.save_model(base_folder+"/Modelos entrenados/Swin v1 Base 3epocas")

In [20]:
# # EVALUAR EL MODELO

test_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/test")


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [21]:
%%time
test_results = trainer.evaluate(test_hf_ds)

print("\n--- Resultados de la evaluación en el dataset de test ---")
print(test_results)


Metric computation result: {'accuracy': 0.899566650390625}

--- Resultados de la evaluación en el dataset de test ---
{'eval_accuracy': 0.899566650390625, 'eval_loss': 0.37863466143608093, 'eval_runtime': 237.91, 'eval_samples_per_second': 137.733, 'eval_steps_per_second': 4.304, 'epoch': 3.0}
CPU times: user 4min 52s, sys: 2.93 s, total: 4min 55s
Wall time: 3min 57s
